## 文本生成

#### 一、需要注意的点：

* 1、`LTP` 使用的是 `cuda` ，在进行分词的时候遇到了内存不足的情况，所以采用 `jieba` 分词。

* 2、使用 `conda` 虚拟环境时，如果爆出 `no jieba` 使用 `canda install jieba` 解决 

#### 二、可能影响模型效果的步骤：

* 1、文本打标签的步骤，比如 `<bos> <eos> <pad>` 等标签。


* 2、此时需要进 `pad_sq` 填充，造成文末很多 `<pad>` 标签进行补齐。


* 3、预测效果不好的原因也会受训练时训练数据 `inputs` 长度的影响。


* 4、过少的 `epoch` 最后生成的文本预测结果会出现重复文字的情况


* 5、神经网络隐藏层的神经元数量和LSTM 隐藏层的熟练对最后模型的学习有影响，神经元少，隐藏层少，最后的预测结果会出现重复文字的情况

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import numpy as np
# from ltp import LTP
import jieba

In [2]:
# Constants
# BOS_TOKEN = "<bos>" ## 句首标记
# EOS_TOKEN = "<eos>" ## 句尾标记
PAD_TOKEN = "<pad>" ## 补齐标记

In [3]:
from collections import defaultdict, Counter

class Vocab:
    def __init__(self, tokens=None):
        self.idx_to_token = list()
        self.token_to_idx = dict()

        if tokens is not None:
            if "<unk>" not in tokens:
                tokens = tokens + ["<unk>"]
            for token in tokens:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
            self.unk = self.token_to_idx['<unk>']

    @classmethod
    def build(cls, text, min_freq=1, reserved_tokens=None):
        token_freqs = defaultdict(int)
        for sentence in text:
            for token in sentence:
                token_freqs[token] += 1
        uniq_tokens = ["<unk>"] + (reserved_tokens if reserved_tokens else [])
        uniq_tokens += [token for token, freq in token_freqs.items() \
                        if freq >= min_freq and token != "<unk>"]
        
        return cls(uniq_tokens)   ## 返回cls 对象，到时候就可以通过这个cls 来调用 Vocab 类中的其他方法
                                  ## 后面调用 build ，并且将返回值设为 vocab = cls(uniq_tokens) ，之后调用vocab，会默认带有uniq_tokens参数
                                 ## 此 uniq_tokens 参数会传给 __init__ 中的 tokens。
    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, token):
        return self.token_to_idx.get(token, self.unk)

    def convert_tokens_to_ids(self, tokens):
        return [self[token] for token in tokens] 

    def convert_ids_to_tokens(self, indices):
        return [self.idx_to_token[index] for index in indices]

#### 添加自定义词典

In [4]:
## 添加自定义词典
jieba.load_userdict('./dict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.812 seconds.
Prefix dict has been built succesfully.


#### 尝试将一段文本分词

In [5]:
## 尝试将一段文本分词
' '.join(jieba.cut('第一章陨落的天才')).split()

['第一章', '陨落', '的', '天才']

#### 加载文本，然后进行分词，写入到分词文件中

In [6]:
## 加载文本，然后进行分词，写入到分词文件中

def text_to_words(path ='./斗破苍穹.txt',path2 ='./nlp_test1.txt' ):
    
    with open(path,encoding='UTF-8',mode='r') as f:
    #     document = f.read()[0:370000].replace(' ','').replace('“','').replace('”','').replace('，','') ## 尝试去除一些标点符号
        document = f.read()[0:370000].replace(' ','')

        document_cut = jieba.cut(document)

        #print  ' '.join(jieba_cut)  //注意：如果打印结果，则分词效果消失，后面的result无法显示
        result = ' '.join(document_cut)
        with open(path2, encoding='UTF-8',mode='w') as f2:
            f2.write(result)

    path = './nlp_test1.txt'
    
    return path

In [7]:
# path= './斗破苍穹_辉哥版.txt'
path= './斗破苍穹.txt'

path = text_to_words(path)

In [8]:
## 查看一下分词文件
with open(path, 'r',encoding='UTF-8') as f:
#     text = f.read()  ## 赋值之后，下面的print 输出就为空了
    print(f.read())

第一章 陨落 的 天才 
 
 “ 斗之力 ， 三段 ！ ” 
 
 望着 测验 魔石碑 上面 闪亮 得 甚至 有些 刺眼 的 五个 大字 ， 少年 面 无表情 ， 唇角 有着 一抹 自嘲 ， 紧握 的 手掌 ， 因为 大力 ， 而 导致 略微 尖锐 的 指甲 深深 的 刺进 了 掌心 之中 ， 带来 一阵阵 钻心 的 疼痛 … 
 
 “ 萧炎 ， 斗之力 ， 三段 ！ 级别 ： 低级 ！ ” 测验 魔石碑 之 旁 ， 一位 中年男子 ， 看 了 一眼 碑 上 所 显示 出来 的 信息 ， 语气 漠然 的 将 之 公布 了 出来 … 
 
 中年男子 话 刚刚 脱口 ， 便是 不出意外 的 在 人头 汹涌 的 广场 上带 起 了 一阵 嘲讽 的 骚动 。 
 
 “ 三段 ？ 嘿嘿 ， 果然 不出 我 所料 ， 这个 “ 天才 ” 这 一年 又是 在 原地踏步 ！ ” 
 
 “ 哎 ， 这 废物 真是 把 家族 的 脸 都 给 丢光 了 。 ” 
 
 “ 要不是 族长 是 他 的 父亲 ， 这种 废物 ， 早就 被 驱赶 出 家族 ， 任其 自生自灭 了 ， 哪 还有 机会 待 在 家族 中 白吃白喝 。 ” 
 
 “ 唉 ， 昔年 那 名闻 乌坦城 的 天才少年 ， 如今 怎么 落魄 成 这般 模样 了 啊 ？ ” 
 
 “ 谁 知道 呢 ， 或许 做 了 什么 亏心事 ， 惹 得 神灵 降怒 了 吧 … ” 
 
 周围 传来 的 不屑 嘲笑 以及 惋惜 轻叹 ， 落 在 那 如 木桩 待 在 原地 的 少年 耳 中 ， 恍如 一根根 利刺 狠狠 的 扎 在 心脏 一般 ， 让 得 少年 呼吸 微微 急促 。 
 
 少年 缓缓 抬起头来 ， 露出 一张 有些 清秀 的 稚嫩 脸庞 ， 漆黑 的 眸子 木然 的 在 周围 那些 嘲讽 的 同龄人 身上 扫 过 ， 少年 嘴角 的 自嘲 ， 似乎 变得 更加 苦涩 了 。 
 
 “ 这些 人 ， 都 如此 刻薄 势力 吗 ？ 或许 是因为 三年 前 他们 曾经 在 自己 面前 露出 过 最 谦卑 的 笑容 ， 所以 ， 如今 想要 讨还 回去 吧 … ” 苦涩 的 一笑 ， 萧炎 落寞 的 转身 ， 安静 的 回到 了 队伍 的 最后 一排 ， 孤单 的 身影 ， 与 周围 的 世界 ， 有些 格格不入

 全神贯注 的 注视 着 药鼎 之内 ， 萧炎 再次 投进 一株 粉红色 的 七叶 花 ， 此花 名为 欲望 花 ， 它 所 散发 出来 的 香味 ， 经常 会 将 附近 的 魔兽


#### lstm 和rnn 进行文本训练的时候不需要去除标点符号和引入停止符，但是在读取数据的时候会有多余的 \n 符号，这里进行过滤

In [9]:
## 测试，将文中的\n 去掉，会应用到 load_reuters 中
# text = []
# with open(path,encoding='UTF-8',mode='r') as f:
#     for i in f.readlines():
#         i = i.replace('\n', '').split()
#         text.append(i)

# print('分词过滤前text len为：',len(text))
# text  = [line for line in text if line != [] and  10 < len(line) ]
# print('分词过滤后text len为：',len(text))

In [10]:
## 测试，将文中的\n 去掉，会应用到 load_data 中
# text = []
# tmp_text = []
# with open(path,encoding='UTF-8',mode='r') as f:
#     for i,content in enumerate(f.readlines()):
#         ##  20 行为一个输入样本
#         if i% 10 == 0:
#             text.append(tmp_text)
#             tmp_text = []
            
#         else:
#             content = content.replace('\n','')
#             tmp_text.extend(content.split())
            

# print('分词过滤前text len为：',len(text))
# text  = [line for line in text if line != []]
# print('分词过滤后text len为：',len(text))

#### 加载数据

In [11]:
## 拿文中的每一段作为一个样本进行训练
def load_words_text(path):
    
    text = []

    with open(path,encoding='UTF-8',mode='r') as f:
        for i in f.readlines():
            i = i.replace('\n', '').split()
            text.append(i)
    text  = [line for line in text if line != [] ]
    

#     vocab = Vocab.build(text, reserved_tokens=[PAD_TOKEN, BOS_TOKEN, EOS_TOKEN])
    vocab = Vocab.build(text, reserved_tokens=[PAD_TOKEN])

#     vocab = Vocab.build(text)

    corpus = [vocab.convert_tokens_to_ids(sentence) for sentence in text]

    return corpus, vocab

In [12]:
# ## 读取文本的时候选用20行做一个样本（不是很必要）
# def load_reuters(path):
    
#     text = []
#     tmp_text = []
    
#     with open(path,encoding='UTF-8',mode='r') as f:
#         for i,content in enumerate(f.readlines()):
#             ##  20 行为一个输入样本
#             if i% 10 == 0:
#                 text.append(tmp_text)
#                 tmp_text = []

#             else:
#                 content = content.replace('\n','')
#                 tmp_text.extend(content.split())
                
#     text  = [line for line in text if line != []]
    
#     # lowercase (optional)
# #     segment = [[word.lower() for word in sentence] for sentence in segment]
# #     vocab = Vocab.build(text, reserved_tokens=[PAD_TOKEN, BOS_TOKEN, EOS_TOKEN])
#     vocab = Vocab.build(text)

#     corpus = [vocab.convert_tokens_to_ids(sentence) for sentence in text]

#     return corpus, vocab

#### 将数据转为 dataloader 数据类型

In [13]:
def get_loader(dataset, batch_size, shuffle=True):
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=dataset.collate_fn,
        shuffle=shuffle
    )
    return data_loader

#### 保存词向量

In [14]:
# def save_pretrained(vocab, embeds, save_path):
#     """
#     Save pretrained token vectors in a unified format, where the first line
#     specifies the `number_of_tokens` and `embedding_dim` followed with all
#     token vectors, one token per line.
#     """
#     with open(save_path, "w") as writer:
#         writer.write(f"{embeds.shape[0]} {embeds.shape[1]}\n")
#         for idx, token in enumerate(vocab.idx_to_token):
#             vec = " ".join(["{:.4f}".format(x) for x in embeds[idx]])
#             writer.write(f"{token} {vec}\n")
#     print(f"Pretrained embeddings saved to: {save_path}")

#### 如果batch_size 为1 ，下面函数的意义不大，如果batch_size 不为1，那么因为序列长度不一致的原因，需要进行补齐，需要用到如下函数

In [15]:
from torch.nn.utils.rnn import pad_sequence

class RnnlmDataset(Dataset):
    def __init__(self, corpus, vocab):
        self.data = []
#         self.bos = vocab[BOS_TOKEN]
#         self.eos = vocab[EOS_TOKEN]
        self.pad = vocab[PAD_TOKEN]
        for sentence in tqdm(corpus, desc="Dataset Construction"):
            # 模型输入：BOS_TOKEN, w_1, w_2, ..., w_n
#             input = [self.bos] + sentence                ## 给数据打上起始标签
            input = sentence[:-1]                          ## 不使用起始标签

            # 模型输出：w_1, w_2, ..., w_n, EOS_TOKEN
#             target = sentence + [self.eos]              ## 给数据打上结束标签
            target = sentence[1:]                         ## 不使用结束标签

            self.data.append((input, target))             

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i]

    def collate_fn(self, examples):
        # 从独立样本集合中构建batch输入输出
        inputs = [torch.tensor(ex[0]) for ex in examples]
        targets = [torch.tensor(ex[1]) for ex in examples]
        # 对batch内的样本进行padding，使其具有相同长度
        ## 即使你设置batch 为1 ，最好也进行补齐，因为操作之后维度会变成2D,不进行补齐维度是1D，在训练模型的时候需要进行一个torch.unse..操作
        inputs = pad_sequence(inputs, batch_first=True, padding_value=self.pad)
        targets = pad_sequence(targets, batch_first=True, padding_value=self.pad)
        
#         inputs = torch.unsqueeze(torch.from_numpy(np.array(inputs)),dim=0)
#         targets = torch.unsqueeze(torch.from_numpy(np.array(targets)),dim=0)

        
        return (inputs, targets)

In [16]:
## 循环神经网络语言模型
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim,num_layers=2):
        super(RNNLM, self).__init__()
        # 词嵌入层
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # 循环神经网络：这里使用LSTM
        self.rnn = nn.LSTM(embedding_dim, hidden_dim,num_layers=num_layers, batch_first=True)
        # 输出层
        self.output = nn.Linear(hidden_dim, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # 计算每一时刻的隐含层表示
        hidden, _ = self.rnn(embeds)
        output = self.output(hidden)

        return output

In [17]:
## 循环神经网络语言模型
# class RNNLM(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim,num_layers=2):
#         super(RNNLM, self).__init__()
#         # 词嵌入层
#         self.embeddings = nn.Embedding(vocab_size, embedding_dim)
#         # 循环神经网络：这里使用LSTM
#         self.rnn = nn.LSTM(embedding_dim, hidden_dim,num_layers=num_layers, batch_first=True)
#         # 输出层
#         self.output = nn.Linear(hidden_dim, vocab_size)

#     def forward(self, inputs):
#         embeds = self.embeddings(inputs)
#         # 计算每一时刻的隐含层表示
#         hidden, _ = self.rnn(embeds)
#         output = self.output(hidden)


#         return output

In [46]:
embedding_dim = 128
hidden_dim = 256
batch_size = 8
num_epoch = 10

# 读取文本数据，构建FFNNLM训练数据集（n-grams）
corpus, vocab = load_words_text(path)
dataset = RnnlmDataset(corpus, vocab)
data_loader = get_loader(dataset, batch_size)

HBox(children=(IntProgress(value=0, description='Dataset Construction', max=6005, style=ProgressStyle(description_width='initial')), HTML(value='')))

In [47]:
len(data_loader)

751

In [48]:
## 交叉熵损失函数
ce_loss  = nn.CrossEntropyLoss()
# 构建RNNLM，并加载至device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RNNLM(len(vocab), embedding_dim, hidden_dim)
# model.to(device)
model

RNNLM(
  (embeddings): Embedding(15307, 128)
  (rnn): LSTM(128, 256, num_layers=2, batch_first=True)
  (output): Linear(in_features=256, out_features=15307, bias=True)
)

In [59]:
# 使用Adam优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(num_epoch):
    total_loss = 0
    for batch in tqdm(data_loader, desc=f"Training Epoch {epoch}"):
#         inputs, targets = [x.to(device) for x in batch]
        inputs, targets = [x for x in batch]

        optimizer.zero_grad()
        output = model(inputs)
        loss = ce_loss(output.view(-1, output.shape[-1]), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()        ## 记录总损失
    print(f"Loss: {total_loss:.2f}")

# save_pretrained(vocab, model.embeddings.weight.data, "rnnlm.vec") ## 保存词向量

HBox(children=(IntProgress(value=0, description='Training Epoch 0', max=751, style=ProgressStyle(description_width='initial')), HTML(value='')))


Loss: 1106.73


HBox(children=(IntProgress(value=0, description='Training Epoch 1', max=751, style=ProgressStyle(description_width='initial')), HTML(value='')))


Loss: 951.66


HBox(children=(IntProgress(value=0, description='Training Epoch 2', max=751, style=ProgressStyle(description_width='initial')), HTML(value='')))

...


Loss: 506.56


HBox(children=(IntProgress(value=0, description='Training Epoch 8', max=751, style=ProgressStyle(description_width='initial')), HTML(value='')))


Loss: 459.10


HBox(children=(IntProgress(value=0, description='Training Epoch 9', max=751, style=ProgressStyle(description_width='initial')), HTML(value='')))


Loss: 418.84


### 预测函数

我们不能单单用一个词来预测下一个词，这样就成了 n-gram 了，所以我们在预测的时候也不能只用一个词来预测下一个。而要用前面的所有作为条件预测下一个。

因为构造的 inputs 和 target 是按照 sentence[:-1] 和 sentence[1:] 来进行构造的。滑动的size 为1，所以对于模型最终的预测结果来说，真正起到预测的只是最后一个词。

text 为输入的文本，如果想要让文本自动的生成，你需要不断的进行预测，第一次预测，将输入的 text 转成 ids 形式，赋值给words_index这是输入样本，输入之后，将预测结果中真正有意义的最后一个词添加到words_index 中，反复进行这样的操作，最后就能进行文本生成。

In [60]:
def predict(model, text, next_words=20,device='cuda'):
      
    words_index = vocab.convert_tokens_to_ids(text)
    ## 将上面的index 转成tensor形式
    words_index = torch.from_numpy(np.array([int(line) for line in words_index if line != 1])).long().to(device) 

    model.eval()     
    model.to(device)
    
    with torch.no_grad():

        for i in range(0, next_words):
            
            y_pred = model(torch.unsqueeze(words_index,dim=0))

            pre_word_index = y_pred.argmax(dim=2)
            words_index = torch.cat([words_index,pre_word_index[0][[-1]]],dim=0)
            
    return words_index

In [61]:
text  = ['你','真的','是','萧炎','吗','?']
text

['你', '真的', '是', '萧炎', '吗', '?']

In [62]:
words_index = vocab.convert_tokens_to_ids(text)
words_index

[579, 1884, 106, 49, 194, 621]

In [63]:
pre_words  = predict(model,text,next_words=350,device='cpu')

In [64]:
# pre_words

In [65]:
''.join(vocab.convert_ids_to_tokens(pre_words))

'你真的是萧炎吗?”心头闪过身子，加列奥嘴角一裂，不敢有些慵懒的咬着手中的古朴卷轴，额头之上，冷汗横流，也有人见到儿这幅窘迫的模样，嘴角微掀，心头忍不住的冷笑。并不比赫蒙小，双掌缓缓摊开，旋即紧紧的盯着萧炎的手掌，旋即冷笑，轻声道：“萧炎哥哥，你…”点了点头，萧炎不再开口，将目光转移到嘴中的美丽女人。后者那略微有些发地感觉。他地斗气。几乎让得他有着大地将之相出来。对着他地歧视。都有自己修炼时间。为了让他能够拒绝地一些兴趣。在他地同伴。也是源源不断与之左右。所以也并不容易。让得他将之强行了头。便是将之一扫而空。旋即一愣。旋即将绿牌看待地面。然后缓缓地行。对着萧炎等人挥了挥手。从睡眠内部地吼出。缓缓地对着那安静地营地之中。顿时胸口猛地发出甩进纳戒。然后再次重重地轰地面。然后暴卷地对着萧炎等去。将石门地布帘声响。顿时。雕刻得紧上在了视线。手中大刀。缓缓地对着萧炎地脑袋。顿时显现在了脸庞之上。顿时。一把从怀中掏出一块碧绿色地牌子。”手掌揉了揉额头。望着那在嘴中缓缓地拳头。萧炎目光森冷地盯着两人手中地身体。缓缓移动。缓缓地对着萧炎踏去。然后握紧营地。将石门堵得拉开。旋即从怀中掏出一张巨大地黑尺。放在桌上。从睡眠中清醒过来地吼出。掌心中对着密林之中行去。他当他与戈'

#### 保存模型

In [58]:
save_path = './lstm_net_epoch10_batch_8'
# 保存整个网络
# torch.save(model2, save_path) 

# 保存网络中的参数, 速度快，占空间少
torch.save(model.state_dict(),save_path)

#### 加载模型

In [ ]:
# model3 = RNNLM(len(vocab), embedding_dim, hidden_dim)
# model3

In [ ]:
#针对上面一般的保存方法，加载的方法分别是：
# model_dict=torch.load(save_path)

# model3.load_state_dict(torch.load(save_path))

## --------------------------------------------------- 调试部分 --------------------------------------------------- 

In [ ]:
vocab.convert_ids_to_tokens(inputs[0])

In [ ]:
words_index = vocab.convert_tokens_to_ids(text)
words_index

In [ ]:
words_index = torch.from_numpy(np.array([int(line) for line in words_index if line != 1])).long().to('cuda')
words_index

In [ ]:
torch.unsqueeze(words_index,dim=0)

In [ ]:
with torch.no_grad():
    y_pred = model(torch.unsqueeze(words_index,dim=0))

In [ ]:
y_pred = y_pred.argmax(dim=2)
y_pred

In [ ]:
vocab.convert_ids_to_tokens([y_pred[0][-1]])

In [ ]:
y_pred[0][[-1]]

In [ ]:
words_index

In [ ]:
words_index = torch.cat([words_index,y_pred[0][[-1]]],dim=0)
words_index

In [ ]:
' '.join(vocab.convert_ids_to_tokens(words_index))

In [ ]:
# ' '.join(vocab.convert_ids_to_tokens(inputs[0]))